In [3]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List, Dict
from collections import defaultdict
import asyncio
import nest_asyncio
nest_asyncio.apply()
import configparser

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.utils.pandas import scols, ohlc_resample
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64, Deal, Order
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel
from qubx.impl.utils import  ccxt_convert_order_info, ccxt_convert_deal_info, ccxt_restore_position_from_deals, ccxt_extract_deals_from_exec
from qubx.utils.pandas import ohlc_resample

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
from qubx.impl.utils import  ccxt_convert_order_info, ccxt_convert_deal_info, ccxt_restore_position_from_deals, ccxt_extract_deals_from_exec
from qubx.core.account import AccountProcessor

In [ ]:
class XXX:
    _processed_trades: Dict[str, List[str]] 

    def __init__(self) -> None:
        self._processed_trades = defaultdict(list)

    def _process_order_report(self, pos: Position, symbol: str, exec_report: Dict[str, Any]):
        # - how to convert to account base currency
        conversion_rate = 1
    
        if pos is not None:
            traded_amnt, realized_pnl, deal_cost = 0, 0, 0
            order = ccxt_convert_order_info(symbol, exec_report)
            deals = ccxt_extract_deals_from_exec(exec_report)

            trades_processed = self._processed_trades[order.id]
            for d in deals:
                if d.id not in trades_processed:
                    trades_processed.append(d.id)
                    realized_pnl += pos.update_position_by_deal(d, conversion_rate) 
                    deal_cost += abs(d.amount) * d.price / conversion_rate
                    traded_amnt += abs(d.amount)

            if traded_amnt != 0:
                logger.info(f"Order {order.id} [{order.status}] -> {order.side} {symbol} {traded_amnt} @ {deal_cost / traded_amnt} -> {realized_pnl:.2f}")
            else:
                logger.info(f"Order {order.id} -> {order.side} {order.quantity} {symbol} -> {order.status}")
        else:
            pass
            # logger.info(str(order))

        if order.status in ['CLOSED', 'CANCELED']:
            self._processed_trades.pop(order.id)

        # Update the active orders list
        if False:
            order_cost = order.quantity * order.price / conversion_rate
            # if order.status in ['CLOSED', 'CANCELED']:
                # print('CLOSED')
                # del self._active_orders[order.id]

            #     # - free locked capital
            #     if order.status == 'CANCELED':
            #         # ------------------------ ---- ------------------------
            #         # ------------------------ TODO ------------------------
            #         # ------------------------ ---- ------------------------
            #         self._locked_capital_in_base -= order_cost
            #         # ------------------------ ---- ------------------------
            # else:
            #     # - lock capital
            #     if order.status in ['NEW', 'OPEN']:
            #         # ------------------------ ---- ------------------------
            #         # ------------------------ TODO ------------------------
            #         # ------------------------ ---- ------------------------
            #         self._locked_capital_in_base += order_cost
            #         # ------------------------ ---- ------------------------

            #     self._active_orders[order.id] = order

In [66]:
buy_RAREUSDT1 = [
    {'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 158473287, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712589685954, 't': -1, 'I': 337411846, 'w': True, 'm': False, 'M': False, 'O': 1712589685954, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'symbol': 'RARE/USDT', 'id': '158473287', 'clientOrderId': 'x-R4BD3S829e799021c990671c96e6dc', 
     'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'lastTradeTimestamp': None, 
     'lastUpdateTimestamp': 1712589685954, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 
     'reduceOnly': None, 'side': 'buy', 'price': None, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 250.0, 'cost': 0.0, 
     'average': None, 'filled': 0.0, 'remaining': 250.0, 'status': 'open', 'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]
buy_RAREUSDT2 = [
    {'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 158473287, 'l': '238.90000000', 'z': '238.90000000', 'L': '0.17850000', 'n': '0.00005426', 'N': 'BNB', 'T': 1712589685954, 't': 19204720, 'I': 337411847, 'w': False, 'm': False, 'M': True, 'O': 1712589685954, 'Z': '42.64365000', 'Y': '42.64365000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'symbol': 'RARE/USDT', 'id': '158473287', 'clientOrderId': 'x-R4BD3S829e799021c990671c96e6dc', 'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'lastTradeTimestamp': 1712589685954, 'lastUpdateTimestamp': 1712589685954, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.1785, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 250.0, 'cost': 42.64365, 'average': 0.1785, 'filled': 238.9, 'remaining': 11.1, 'status': 'open', 'fee': {'cost': 5.426e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 158473287, 'l': '238.90000000', 'z': '238.90000000', 'L': '0.17850000', 'n': '0.00005426', 'N': 'BNB', 'T': 1712589685954, 't': 19204720, 'I': 337411847, 'w': False, 'm': False, 'M': True, 'O': 1712589685954, 'Z': '42.64365000', 'Y': '42.64365000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'symbol': 'RARE/USDT', 'id': '19204720', 'order': '158473287', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 0.1785, 'amount': 238.9, 'cost': 42.64365, 'fee': {'cost': 5.426e-05, 'currency': 'BNB'}, 'fees': [{'cost': 5.426e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
# 2024-04-08 18:21:25.770 [ ℹ️ ] Order 158473287 -> BUY RAREUSDT 238.9 @ 0.1785 -> 0.00 USDT
]
buy_RAREUSDT3 = [
    {'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 158473287, 'l': '11.10000000', 'z': '250.00000000', 'L': '0.17850000', 'n': '0.00000252', 'N': 'BNB', 'T': 1712589685954, 't': 19204721, 'I': 337411849, 'w': False, 'm': False, 'M': True, 'O': 1712589685954, 'Z': '44.62500000', 'Y': '1.98135000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'symbol': 'RARE/USDT', 'id': '158473287', 'clientOrderId': 'x-R4BD3S829e799021c990671c96e6dc', 'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'lastTradeTimestamp': 1712589685954, 'lastUpdateTimestamp': 1712589685954, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.1785, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 250.0, 'cost': 44.625, 'average': 0.1785, 'filled': 250.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 5.678e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 158473287, 'l': '238.90000000', 'z': '238.90000000', 'L': '0.17850000', 'n': '0.00005426', 'N': 'BNB', 'T': 1712589685954, 't': 19204720, 'I': 337411847, 'w': False, 'm': False, 'M': True, 'O': 1712589685954, 'Z': '42.64365000', 'Y': '42.64365000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'symbol': 'RARE/USDT', 'id': '19204720', 'order': '158473287', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 0.1785, 'amount': 238.9, 'cost': 42.64365, 'fee': {'cost': 5.426e-05, 'currency': 'BNB'}, 'fees': [{'cost': 5.426e-05, 'currency': 'BNB'}]}, {'info': {'e': 'executionReport', 'E': 1712589685955, 's': 'RAREUSDT', 'c': 'x-R4BD3S829e799021c990671c96e6dc', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '250.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 158473287, 'l': '11.10000000', 'z': '250.00000000', 'L': '0.17850000', 'n': '0.00000252', 'N': 'BNB', 'T': 1712589685954, 't': 19204721, 'I': 337411849, 'w': False, 'm': False, 'M': True, 'O': 1712589685954, 'Z': '44.62500000', 'Y': '1.98135000', 'Q': '0.00000000', 'W': 1712589685954, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712589685954, 'datetime': '2024-04-08T15:21:25.954Z', 'symbol': 'RARE/USDT', 'id': '19204721', 'order': '158473287', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 0.1785, 'amount': 11.1, 'cost': 1.98135, 'fee': {'cost': 2.52e-06, 'currency': 'BNB'}, 'fees': [{'cost': 2.52e-06, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
# 2024-04-08 18:21:25.772 [ ℹ️ ] Order 158473287 -> BUY RAREUSDT 250.0 @ 0.1785 -> 0.00 USDT
]

sell_RAREUSDT1 = [
		{'info': {'e': 'executionReport', 'E': 1712647798241, 's': 'RAREUSDT', 'c': 'CMFIRanking_RAREUSDT_17126474946', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '151.70000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 158633865, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712647798240, 't': -1, 'I': 337752039, 'w': True, 'm': False, 'M': False, 'O': 1712647798240, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712647798240, 'V': 'EXPIRE_MAKER'}, 'symbol': 'RARE/USDT', 'id': '158633865', 'clientOrderId': 'CMFIRanking_RAREUSDT_17126474946', 'timestamp': 1712647798240, 'datetime': '2024-04-09T07:29:58.240Z', 'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1712647798240, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'sell', 'price': None, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 151.7, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 151.7, 'status': 'open', 'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]

sell_RAREUSDT2 = [
		{'info': {'e': 'executionReport', 'E': 1712647798241, 's': 'RAREUSDT', 'c': 'CMFIRanking_RAREUSDT_17126474946', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '151.70000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 158633865, 'l': '151.70000000', 'z': '151.70000000', 'L': '0.17300000', 'n': '0.00003413', 'N': 'BNB', 'T': 1712647798240, 't': 19224199, 'I': 337752040, 'w': False, 'm': False, 'M': True, 'O': 1712647798240, 'Z': '26.24410000', 'Y': '26.24410000', 'Q': '0.00000000', 'W': 1712647798240, 'V': 'EXPIRE_MAKER'}, 
   'symbol': 'RARE/USDT', 'id': '158633865', 'clientOrderId': 'CMFIRanking_RAREUSDT_17126474946', 
   'timestamp': 1712647798240, 'datetime': '2024-04-09T07:29:58.240Z', 
   'lastTradeTimestamp': 1712647798240, 'lastUpdateTimestamp': 1712647798240, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 
   'reduceOnly': None, 'side': 'sell', 'price': 0.173, 'stopPrice': 0.0, 'triggerPrice': 0.0, 
   'amount': 151.7, 'cost': 26.2441, 'average': 0.173, 'filled': 151.7, 'remaining': 0.0, 'status': 'closed', 
   'fee': {'cost': 3.413e-05, 'currency': 'BNB'}, 
   'trades': [
       {
           	'info': {'e': 'executionReport', 'E': 1712647798241, 's': 'RAREUSDT', 'c': 'CMFIRanking_RAREUSDT_17126474946', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '151.70000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 158633865, 'l': '151.70000000', 'z': '151.70000000', 'L': '0.17300000', 'n': '0.00003413', 'N': 'BNB', 'T': 1712647798240, 't': 19224199, 'I': 337752040, 'w': False, 'm': False, 'M': True, 'O': 1712647798240, 'Z': '26.24410000', 'Y': '26.24410000', 'Q': '0.00000000', 'W': 1712647798240, 'V': 'EXPIRE_MAKER'}, 
            'timestamp': 1712647798240, 'datetime': '2024-04-09T07:29:58.240Z', 
            'symbol': 'RARE/USDT', 'id': '19224199', 'order': '158633865', 'type': 'market', 'takerOrMaker': 'taker', 
            'side': 'sell', 'price': 0.173, 'amount': 151.7, 'cost': 26.2441, 
            'fee': {'cost': 3.413e-05, 'currency': 'BNB'}, 
            'fees': [{'cost': 3.413e-05, 'currency': 'BNB'}]
    	}
    ], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]

execs_SUPERUSDT1 = [
	{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '42.00000000', 'z': '42.00000000', 'L': '1.20090000', 'n': '0.00006554', 'N': 'BNB', 'T': 1712647798496, 't': 43747315, 'I': 1068506586, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '50.43780000', 'Y': '50.43780000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'symbol': 'SUPER/USDT', 'id': '512572947', 'clientOrderId': 'CMFIRanking_SUPERUSDT_17126474947', 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'lastTradeTimestamp': 1712647798496, 'lastUpdateTimestamp': 1712647798496, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 1.2009, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 69.0, 'cost': 50.4378, 'average': 1.2009, 'filled': 42.0, 'remaining': 27.0, 'status': 'open', 'fee': {'cost': 6.554e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '42.00000000', 'z': '42.00000000', 'L': '1.20090000', 'n': '0.00006554', 'N': 'BNB', 'T': 1712647798496, 't': 43747315, 'I': 1068506586, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '50.43780000', 'Y': '50.43780000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747315', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.2009, 'amount': 42.0, 'cost': 50.4378, 'fee': {'cost': 6.554e-05, 'currency': 'BNB'}, 'fees': [{'cost': 6.554e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]

execs_SUPERUSDT2 = [
	{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '7.00000000', 'z': '49.00000000', 'L': '1.20100000', 'n': '0.00001092', 'N': 'BNB', 'T': 1712647798496, 't': 43747316, 'I': 1068506588, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '58.84480000', 'Y': '8.40700000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'symbol': 'SUPER/USDT', 'id': '512572947', 'clientOrderId': 'CMFIRanking_SUPERUSDT_17126474947', 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'lastTradeTimestamp': 1712647798496, 'lastUpdateTimestamp': 1712647798496, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 1.2009142857142858, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 69.0, 'cost': 58.8448, 'average': 1.2009142857142858, 'filled': 49.0, 'remaining': 20.0, 'status': 'open', 'fee': {'cost': 7.646e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '42.00000000', 'z': '42.00000000', 'L': '1.20090000', 'n': '0.00006554', 'N': 'BNB', 'T': 1712647798496, 't': 43747315, 'I': 1068506586, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '50.43780000', 'Y': '50.43780000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747315', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.2009, 'amount': 42.0, 'cost': 50.4378, 'fee': {'cost': 6.554e-05, 'currency': 'BNB'}, 'fees': [{'cost': 6.554e-05, 'currency': 'BNB'}]}, {'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '7.00000000', 'z': '49.00000000', 'L': '1.20100000', 'n': '0.00001092', 'N': 'BNB', 'T': 1712647798496, 't': 43747316, 'I': 1068506588, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '58.84480000', 'Y': '8.40700000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747316', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.201, 'amount': 7.0, 'cost': 8.407, 'fee': {'cost': 1.092e-05, 'currency': 'BNB'}, 'fees': [{'cost': 1.092e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]

execs_SUPERUSDT3 = [
	{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 512572947, 'l': '20.00000000', 'z': '69.00000000', 'L': '1.20100000', 'n': '0.00003120', 'N': 'BNB', 'T': 1712647798496, 't': 43747317, 'I': 1068506590, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '82.86480000', 'Y': '24.02000000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'symbol': 'SUPER/USDT', 'id': '512572947', 'clientOrderId': 'CMFIRanking_SUPERUSDT_17126474947', 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'lastTradeTimestamp': 1712647798496, 'lastUpdateTimestamp': 1712647798496, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 1.2009391304347825, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 69.0, 'cost': 82.8648, 'average': 1.2009391304347825, 'filled': 69.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 0.00010766, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '42.00000000', 'z': '42.00000000', 'L': '1.20090000', 'n': '0.00006554', 'N': 'BNB', 'T': 1712647798496, 't': 43747315, 'I': 1068506586, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '50.43780000', 'Y': '50.43780000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747315', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.2009, 'amount': 42.0, 'cost': 50.4378, 'fee': {'cost': 6.554e-05, 'currency': 'BNB'}, 'fees': [{'cost': 6.554e-05, 'currency': 'BNB'}]}, {'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'PARTIALLY_FILLED', 'r': 'NONE', 'i': 512572947, 'l': '7.00000000', 'z': '49.00000000', 'L': '1.20100000', 'n': '0.00001092', 'N': 'BNB', 'T': 1712647798496, 't': 43747316, 'I': 1068506588, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '58.84480000', 'Y': '8.40700000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747316', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.201, 'amount': 7.0, 'cost': 8.407, 'fee': {'cost': 1.092e-05, 'currency': 'BNB'}, 'fees': [{'cost': 1.092e-05, 'currency': 'BNB'}]}, {'info': {'e': 'executionReport', 'E': 1712647798496, 's': 'SUPERUSDT', 'c': 'CMFIRanking_SUPERUSDT_17126474947', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '69.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 512572947, 'l': '20.00000000', 'z': '69.00000000', 'L': '1.20100000', 'n': '0.00003120', 'N': 'BNB', 'T': 1712647798496, 't': 43747317, 'I': 1068506590, 'w': False, 'm': False, 'M': True, 'O': 1712647798496, 'Z': '82.86480000', 'Y': '24.02000000', 'Q': '0.00000000', 'W': 1712647798496, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712647798496, 'datetime': '2024-04-09T07:29:58.496Z', 'symbol': 'SUPER/USDT', 'id': '43747317', 'order': '512572947', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 1.201, 'amount': 20.0, 'cost': 24.02, 'fee': {'cost': 3.12e-05, 'currency': 'BNB'}, 'fees': [{'cost': 3.12e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},
]

execs_ACA = [
    [{'info': {'e': 'executionReport', 'E': 1712231084293, 's': 'ACAUSDT', 'c': 'x-R4BD3S8238819a3237ee3b4e56266c', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 149024288, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712231084293, 't': -1, 'I': 315492989, 'w': True, 'm': False, 'M': False, 'O': 1712231084293, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712231084293, 'V': 'EXPIRE_MAKER'}, 'symbol': 'ACA/USDT', 'id': '149024288', 'clientOrderId': 'x-R4BD3S8238819a3237ee3b4e56266c', 'timestamp': 1712231084293, 'datetime': '2024-04-04T11:44:44.293Z', 'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1712231084293, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': None, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 50.0, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 50.0, 'status': 'open', 'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},

    {'info': {'e': 'executionReport', 'E': 1712231084293, 's': 'ACAUSDT', 'c': 'x-R4BD3S8238819a3237ee3b4e56266c', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149024288, 'l': '50.00000000', 'z': '50.00000000', 'L': '0.16120000', 'n': '0.00001027', 'N': 'BNB', 'T': 1712231084293, 't': 17285573, 'I': 315492990, 'w': False, 'm': False, 'M': True, 'O': 1712231084293, 'Z': '8.06000000', 'Y': '8.06000000', 'Q': '0.00000000', 'W': 1712231084293, 'V': 'EXPIRE_MAKER'}, 'symbol': 'ACA/USDT', 'id': '149024288', 'clientOrderId': 'x-R4BD3S8238819a3237ee3b4e56266c', 'timestamp': 1712231084293, 'datetime': '2024-04-04T11:44:44.293Z', 'lastTradeTimestamp': 1712231084293, 'lastUpdateTimestamp': 1712231084293, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.1612, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 50.0, 'cost': 8.06, 'average': 0.1612, 'filled': 50.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 1.027e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712231084293, 's': 'ACAUSDT', 'c': 'x-R4BD3S8238819a3237ee3b4e56266c', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149024288, 'l': '50.00000000', 'z': '50.00000000', 'L': '0.16120000', 'n': '0.00001027', 'N': 'BNB', 'T': 1712231084293, 't': 17285573, 'I': 315492990, 'w': False, 'm': False, 'M': True, 'O': 1712231084293, 'Z': '8.06000000', 'Y': '8.06000000', 'Q': '0.00000000', 'W': 1712231084293, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712231084293, 'datetime': '2024-04-04T11:44:44.293Z', 'symbol': 'ACA/USDT', 'id': '17285573', 'order': '149024288', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'buy', 'price': 0.1612, 'amount': 50.0, 'cost': 8.06, 'fee': {'cost': 1.027e-05, 'currency': 'BNB'}, 'fees': [{'cost': 1.027e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},

    {'info': {'e': 'executionReport', 'E': 1712231324395, 's': 'ACAUSDT', 'c': 'x-ZQ3RVWN66045debc8d86be46f76afa', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 149024642, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712231324394, 't': -1, 'I': 315493717, 'w': True, 'm': False, 'M': False, 'O': 1712231324394, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712231324394, 'V': 'EXPIRE_MAKER'}, 'symbol': 'ACA/USDT', 'id': '149024642', 'clientOrderId': 'x-ZQ3RVWN66045debc8d86be46f76afa', 'timestamp': 1712231324394, 'datetime': '2024-04-04T11:48:44.394Z', 'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1712231324394, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'sell', 'price': None, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 50.0, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 50.0, 'status': 'open', 'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},

    {'info': {'e': 'executionReport', 'E': 1712231324395, 's': 'ACAUSDT', 'c': 'x-ZQ3RVWN66045debc8d86be46f76afa', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149024642, 'l': '50.00000000', 'z': '50.00000000', 'L': '0.16060000', 'n': '0.00001026', 'N': 'BNB', 'T': 1712231324394, 't': 17285596, 'I': 315493718, 'w': False, 'm': False, 'M': True, 'O': 1712231324394, 'Z': '8.03000000', 'Y': '8.03000000', 'Q': '0.00000000', 'W': 1712231324394, 'V': 'EXPIRE_MAKER'}, 'symbol': 'ACA/USDT', 'id': '149024642', 'clientOrderId': 'x-ZQ3RVWN66045debc8d86be46f76afa', 'timestamp': 1712231324394, 'datetime': '2024-04-04T11:48:44.394Z', 'lastTradeTimestamp': 1712231324394, 'lastUpdateTimestamp': 1712231324394, 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'sell', 'price': 0.1606, 'stopPrice': 0.0, 'triggerPrice': 0.0, 'amount': 50.0, 'cost': 8.03, 'average': 0.1606, 'filled': 50.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 1.026e-05, 'currency': 'BNB'}, 'trades': [{'info': {'e': 'executionReport', 'E': 1712231324395, 's': 'ACAUSDT', 'c': 'x-ZQ3RVWN66045debc8d86be46f76afa', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '50.00000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149024642, 'l': '50.00000000', 'z': '50.00000000', 'L': '0.16060000', 'n': '0.00001026', 'N': 'BNB', 'T': 1712231324394, 't': 17285596, 'I': 315493718, 'w': False, 'm': False, 'M': True, 'O': 1712231324394, 'Z': '8.03000000', 'Y': '8.03000000', 'Q': '0.00000000', 'W': 1712231324394, 'V': 'EXPIRE_MAKER'}, 'timestamp': 1712231324394, 'datetime': '2024-04-04T11:48:44.394Z', 'symbol': 'ACA/USDT', 'id': '17285596', 'order': '149024642', 'type': 'market', 'takerOrMaker': 'taker', 'side': 'sell', 'price': 0.1606, 'amount': 50.0, 'cost': 8.03, 'fee': {'cost': 1.026e-05, 'currency': 'BNB'}, 'fees': [{'cost': 1.026e-05, 'currency': 'BNB'}]}], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},

    {
    'info': {
        'e': 'executionReport', 'E': 1712231523596, 's': 'ACAUSDT', 'c': 'x-R4BD3S82e78e232b573fe3c2a48153', 'S': 'BUY', 'o': 'LIMIT', 'f': 'GTC', 'q': '51.00000000', 'p': '0.16110000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 149025236, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712231523596, 't': -1, 'I': 315494940, 'w': True, 'm': False, 'M': False, 'O': 1712231523596, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712231523596, 'V': 'EXPIRE_MAKER'
    }, 
    'symbol': 'ACA/USDT', 
    'id': '149025236', 'clientOrderId': 'x-R4BD3S82e78e232b573fe3c2a48153', 
    'timestamp': 1712231523596, 'datetime': '2024-04-04T11:52:03.596Z', 
    'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1712231523596, 
    'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.1611, 'stopPrice': 0.0, 
    'triggerPrice': 0.0, 'amount': 51.0, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 51.0, 
    'status': 'open', 'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None
    },

    {
    'info': {
        'e': 'executionReport', 'E': 1712231530098, 's': 'ACAUSDT', 'c': 'x-R4BD3S82e78e232b573fe3c2a48153', 'S': 'BUY', 'o': 'LIMIT', 'f': 'GTC', 'q': '51.00000000', 'p': '0.16110000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149025236, 'l': '51.00000000', 'z': '51.00000000', 'L': '0.16110000', 'n': '0.00001044', 'N': 'BNB', 'T': 1712231530082, 't': 17285628, 'I': 315494958, 'w': False, 'm': True, 'M': True, 'O': 1712231523596, 'Z': '8.21610000', 'Y': '8.21610000', 'Q': '0.00000000', 'W': 1712231523596, 'V': 'EXPIRE_MAKER'
    }, 
    'symbol': 'ACA/USDT', 
    'id': '149025236', 'clientOrderId': 'x-R4BD3S82e78e232b573fe3c2a48153', 
    'timestamp': 1712231523596, 
    'datetime': '2024-04-04T11:52:03.596Z', 
    'lastTradeTimestamp': 1712231530082, 
    'lastUpdateTimestamp': 1712231530082, 
    'type': 'limit', 
    'timeInForce': 'GTC', 
    'postOnly': False, 
    'reduceOnly': None, 
    'side': 'buy', 
    'price': 0.1611, 
    'stopPrice': 0.0, 
    'triggerPrice': 0.0, 
    'amount': 51.0, 
    'cost': 8.2161, 
    'average': 0.1611, 
    'filled': 51.0, 
    'remaining': 0.0, 
    'status': 'closed', 
    'fee': {'cost': 1.044e-05, 'currency': 'BNB'}, 
    'trades': [
        {
            'info': {
                'e': 'executionReport', 'E': 1712231530098, 's': 'ACAUSDT', 'c': 'x-R4BD3S82e78e232b573fe3c2a48153', 'S': 'BUY', 'o': 'LIMIT', 'f': 'GTC', 'q': '51.00000000', 'p': '0.16110000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 149025236, 'l': '51.00000000', 'z': '51.00000000', 'L': '0.16110000', 'n': '0.00001044', 'N': 'BNB', 'T': 1712231530082, 't': 17285628, 'I': 315494958, 'w': False, 'm': True, 'M': True, 'O': 1712231523596, 'Z': '8.21610000', 'Y': '8.21610000', 'Q': '0.00000000', 'W': 1712231523596, 'V': 'EXPIRE_MAKER'
            }, 
            'timestamp': 1712231530082, 'datetime': '2024-04-04T11:52:10.082Z', 
            'symbol': 'ACA/USDT', 
            'id': '17285628', 'order': '149025236', 
            'type': 'limit', 
            'takerOrMaker': 'maker', 
            'side': 'buy', 
            'price': 0.1611, 
            'amount': 51.0, 'cost': 8.2161, 
            'fee': {'cost': 1.044e-05, 'currency': 'BNB'}, 'fees': [{'cost': 1.044e-05, 'currency': 'BNB'}]
        }
    ], 
    'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None},

    {
    'info': {'e': 'executionReport', 'E': 1712236270257, 's': 'ACAUSDT', 'c': 'JzCqzi9q0LSiyWH8HJuYtQ', 'S': 'BUY', 'o': 'LIMIT', 'f': 'GTC', 'q': '50.00000000', 'p': '0.10000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': 'x-R4BD3S82d0c6eae956a555dbab1b26', 'x': 'CANCELED', 'X': 'CANCELED', 'r': 'NONE', 'i': 149034846, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1712236270256, 't': -1, 'I': 315514865, 'w': False, 'm': False, 'M': False, 'O': 1712236254516, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1712236254516, 'V': 'EXPIRE_MAKER'}, 
    'symbol': 'ACA/USDT', 
    'id': '149034846', 'clientOrderId': 'x-R4BD3S82d0c6eae956a555dbab1b26',
    'timestamp': 1712236254516, 'datetime': '2024-04-04T13:10:54.516Z', 
    'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1712236270256, 
    'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'buy', 'price': 0.1, 'stopPrice': 0.0, 
    'triggerPrice': 0.0, 'amount': 50.0, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 50.0, 'status': 'canceled', 
    'fee': None, 'trades': [], 'fees': [], 'takeProfitPrice': None, 'stopLossPrice': None
    }]]

In [119]:
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64, Deal, Order


In [126]:
acc = AccountProcessor('USDT', 100, 0)
acc.attach_positions(
    Position(lookup.find_symbol('BINANCE', 'RAREUSDT'), lookup.find_fees('binance', 'vip0_bnb')), # type: ignore
    Position(lookup.find_symbol('BINANCE', 'SUPERUSDT'), lookup.find_fees('binance', 'vip0_bnb')), # type: ignore
    Position(lookup.find_symbol('BINANCE', 'ACAUSDT'), lookup.find_fees('binance', 'vip0_bnb')), # type: ignore
)

for exs in [
    *execs_ACA, 
    buy_RAREUSDT1, buy_RAREUSDT2, buy_RAREUSDT3, sell_RAREUSDT1, sell_RAREUSDT2,
    execs_SUPERUSDT1, execs_SUPERUSDT2, execs_SUPERUSDT3
]:
    for report in exs:
        symbol = report['info']['s']
        order = ccxt_convert_order_info(symbol, report)
        deals = ccxt_extract_deals_from_exec(report)
        acc.process_deals(symbol, deals)
        acc.process_order(order)

print('- ' * 50)
pd.DataFrame.from_dict(acc.positions_report()).T

2024-04-09 15:09:21.840 [ ℹ️ ] Order 149024288 MARKET BUY 50.0 of ACAUSDT -> OPEN
2024-04-09 15:09:21.841 [ ℹ️ ]   ::  traded 50.0 for ACAUSDT @ 0.1612 -> 0.00
2024-04-09 15:09:21.841 [ ℹ️ ] Order 149024288 MARKET BUY 50.0 of ACAUSDT -> CLOSED
2024-04-09 15:09:21.842 [ ℹ️ ] Order 149024642 MARKET SELL 50.0 of ACAUSDT -> OPEN
2024-04-09 15:09:21.842 [ ℹ️ ]   ::  traded -50.0 for ACAUSDT @ 0.1606 -> -0.03
2024-04-09 15:09:21.843 [ ℹ️ ] Order 149024642 MARKET SELL 50.0 of ACAUSDT -> CLOSED
2024-04-09 15:09:21.845 [ ℹ️ ] Order 149025236 LIMIT BUY 51.0 of ACAUSDT -> OPEN
2024-04-09 15:09:21.845 [ ℹ️ ]   ::  traded 51.0 for ACAUSDT @ 0.1611 -> 0.00
2024-04-09 15:09:21.846 [ ℹ️ ] Order 149025236 LIMIT BUY 51.0 of ACAUSDT -> CLOSED
2024-04-09 15:09:21.846 [ ℹ️ ] Order 149034846 LIMIT BUY 50.0 of ACAUSDT -> CANCELED
2024-04-09 15:09:21.847 [ ℹ️ ] Order 158473287 MARKET BUY 250.0 of RAREUSDT -> OPEN
2024-04-09 15:09:21.848 [ ℹ️ ]   ::  traded 238.9 for RAREUSDT @ 0.1785 -> 0.00
2024-04-09 15:09:

,Qty,Price,PnL,MktValue
RAREUSDT,98.3,0.1784,-1.3500,17.0059
SUPERUSDT,69.0,1.2009,0.0069,82.8690
ACAUSDT,51.0,0.1610,-0.0249,8.2161


In [113]:
acc.get_capital()

250.0

In [59]:
acc._active_orders, acc._processed_trades

({}, defaultdict(list, {}))